# 증식을 사용안한 코드

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

train_dir = '../data/cat_dog_full/train'
validation_dir = '../data/cat_dog_full/validation'

# ImageDataGenerator 생성
# train data 생성 ImageDataGenerator, validation data 생성 ImageDataGenerator 각각 생성
# 정규화(MinMaxScaling 이용)
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,                # target directory
    classes=['cats', 'dogs'], # 순서대로 0,1..., classes 속성을 생략하면 폴더 순서대로 label이 결정
    target_size=(150,150),    # 일반적으로 정방향으로 resize, CNN구조에 따라 이 값은 달라진다
    batch_size=100,          # 한번에 500개 이미지 가져와서 변환, label에 상관 없이 가져온다
    class_mode='binary'       # 다중분류면 categorical(default)
                              # 'sparse도 이용가능 - onehot처리 안하고 사용
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,               
    classes=['cats', 'dogs'], 
    target_size=(150,150),   
    batch_size=100,            
    class_mode='binary'
)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), 
                 activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))

print(model.summary())

model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# steps_per_epoch : 몇번 뽑아야 1 epoch이 되는가
history = model.fit(train_generator, steps_per_epoch=140, epochs=30, 
                    validation_data=validation_generator,
                    validation_steps=60)

In [ ]:
# history 객체 확인
print(history.history.keys())

train_acc = history.history['accuracy']
validation_acc = history.history['val_accuracy']

train_loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.plot(train_loss, color='r', label='training')
plt.plot(validation_loss, color='b', label='validation')
plt.legend()
plt.show()